# taq_self_response_day_responses_event_shift_data

#### Juan Camilo Henao Londono - 02.07.2019
#### AG Guhr - Universitaet Duisburg-Essen

In the following [link]() can be seen the plot of the function's results

In [1]:
# Modules

import numpy as np
import os

import pickle

import taq_data_tools

__tau__ = 1000

In [2]:
def taq_self_response_day_responses_event_shift_data(ticker, date, shift, *,
                                                     tau='off'):
    """
    Obtain the self response function using the midpoint price returns
    and trade signs of the ticker during different time lags. Return an
    array with the self response for a day and an array of the number of
    trades for each value of tau.
        :param ticker: string of the abbreviation of the midpoint stock to
         be analized (i.e. 'AAPL')
        :param date: string with the date of the data to be extracted
         (i.e. '2008-01-02')
    """

    date_sep = date.split('-')

    year = date_sep[0]
    month = date_sep[1]
    day = date_sep[2]

    function_name = taq_self_response_day_responses_event_shift_data.__name__
    taq_data_tools.taq_function_header_print_data(function_name, ticker,
                                                  ticker, year, month, day)

    try:

        # Load data
        midpoint_i = pickle.load(open(''.join((
                '../../taq_data/article_reproduction_data_{1}/taq_midpoint'
                + '_full_time_data/taq_midpoint_full_time_data_midpoint_{1}'
                + '{2}{3}_{0}.pickle').split())
                .format(ticker, year, month, day), 'rb'))
        time_t, _, trade_sign_i = pickle.load(open("".join((
                '../../taq_data/responses_event_shift_data_{1}/taq_trade'
                + '_signs_responses_event_shift_data/taq_trade_signs'
                + '_responses_event_shift_data_{1}{2}{3}_{0}.pickle')
                .split())
                .format(ticker, year, month, day), 'rb'))
        # As the data is loaded from the original reproduction data from the
        # article, the data have a shift of 1 second. To correct this I made
        # both data to have the same time [34801, 56999]
        midpoint_i = midpoint_i[1:]
        time_m = np.array(range(34801, 57000))

        assert not np.sum(trade_sign_i == 0)
        assert not np.sum(midpoint_i == 0)

        # Array of the average of each tau. 10^3 s used by Wang
        self_response_tau = np.zeros(__tau__)
        num = np.zeros(__tau__)

        # Calculating the midpoint log return and the self response function

        midpoint_t = 0. * trade_sign_i

        for t_idx, t_val in enumerate(time_m):
            condition = time_t == t_val
            len_c = np.sum(condition)
            midpoint_t[condition] = midpoint_i[t_idx] * np.ones(len_c)

        assert not np.sum(midpoint_t == 0)

        # Depending on the tau value
        if (tau == 'off'):
            for tau_idx in range(__tau__):

                if (shift != 0):
                    midpoint_shift = midpoint_t[:-shift]
                    trade_sign_shift = trade_sign_i[shift:]
                else:
                    midpoint_shift = midpoint_t
                    trade_sign_shift = trade_sign_i

                trade_sign_tau = trade_sign_shift[:-tau_idx-1]
                trade_sign_no_0_len = len(trade_sign_tau[trade_sign_tau != 0])
                num[tau_idx] = trade_sign_no_0_len
                # Obtain the midpoint log return. Displace the numerator tau
                # values to the right and compute the return

                # Midpoint price returns
                log_return_sec = (midpoint_shift[tau_idx + 1:]
                                  - midpoint_shift[:-tau_idx - 1]) \
                    / midpoint_shift[:-tau_idx - 1]

                # Obtain the self response value
                if (trade_sign_no_0_len != 0):
                    product = log_return_sec * trade_sign_tau
                    self_response_tau[tau_idx] = np.sum(product)

        elif (tau == 'on'):
            for tau_idx in range(__tau__):

                midpoint_shift = midpoint_t[:-(tau_idx // 2) - 1]
                trade_sign_shift = trade_sign_i[tau_idx // 2 + 1:]

                assert len(midpoint_shift) == len(trade_sign_shift)

                trade_sign_tau = trade_sign_shift[:-tau_idx-1]
                trade_sign_no_0_len = len(trade_sign_tau[trade_sign_tau != 0])
                num[tau_idx] = trade_sign_no_0_len
                # Obtain the midpoint log return. Displace the numerator tau
                # values to the right and compute the return

                # Midpoint price returns
                log_return_sec = (midpoint_shift[tau_idx + 1:]
                                  - midpoint_shift[:-tau_idx - 1]) \
                    / midpoint_shift[:-tau_idx - 1]

                # Obtain the self response value
                if (trade_sign_no_0_len != 0):
                    product = log_return_sec * trade_sign_tau
                    self_response_tau[tau_idx] = np.sum(product)

        return self_response_tau, num

    except FileNotFoundError as e:
        print('No data')
        print(e)
        print()
        return None

In [3]:
# Test

tickers = ['AAPL', 'MSFT']
dates = []

year = '2008'
month = '03'
days = ['10', '11', '12', '13', '14']

for day in days:
    dates.append(year + '-' + month + '-' + day)

for ticker in tickers:
    for date in dates:
        taq_self_response_day_responses_event_shift_data(ticker, date, 1)

TAQ data
taq_self_response_day_responses_event_shift_data
Processing data for the stock AAPL the 2008.03.10
TAQ data
taq_self_response_day_responses_event_shift_data
Processing data for the stock AAPL the 2008.03.11
TAQ data
taq_self_response_day_responses_event_shift_data
Processing data for the stock AAPL the 2008.03.12
TAQ data
taq_self_response_day_responses_event_shift_data
Processing data for the stock AAPL the 2008.03.13
TAQ data
taq_self_response_day_responses_event_shift_data
Processing data for the stock AAPL the 2008.03.14
TAQ data
taq_self_response_day_responses_event_shift_data
Processing data for the stock MSFT the 2008.03.10
TAQ data
taq_self_response_day_responses_event_shift_data
Processing data for the stock MSFT the 2008.03.11
TAQ data
taq_self_response_day_responses_event_shift_data
Processing data for the stock MSFT the 2008.03.12
TAQ data
taq_self_response_day_responses_event_shift_data
Processing data for the stock MSFT the 2008.03.13
TAQ data
taq_self_response_d

In [8]:
x, y = taq_self_response_day_responses_event_shift_data(ticker, '2008-01-02', '', tau='on')

TAQ data
taq_self_response_day_responses_event_shift_data
Processing data for the stock MSFT the 2008.01.02


In [9]:
x

array([ 0.50321579,  0.78126397,  1.31811724,  1.56935673,  2.10137969,
        2.33945932,  2.87997136,  3.09517092,  3.62864671,  3.85215298,
        4.38844578,  4.59569543,  5.12436002,  5.31706769,  5.84658309,
        6.03622081,  6.55247691,  6.72996193,  7.24735034,  7.41665967,
        7.91396144,  8.06122877,  8.54858532,  8.6980668 ,  9.17216173,
        9.32108513,  9.7814032 ,  9.91122818, 10.36643194, 10.48320783,
       10.93307394, 11.05635285, 11.49013484, 11.61969461, 12.03114308,
       12.1446109 , 12.53092563, 12.63784249, 13.02108856, 13.11898586,
       13.50532499, 13.60198333, 13.96403889, 14.0517259 , 14.39946731,
       14.48114169, 14.81051612, 14.88372302, 15.19391156, 15.25894456,
       15.56291993, 15.62813253, 15.91564616, 15.97975206, 16.26135717,
       16.30597346, 16.57228927, 16.62331125, 16.88060464, 16.9183483 ,
       17.15239992, 17.18705068, 17.41996637, 17.45770787, 17.66131952,
       17.68892909, 17.8702866 , 17.89229383, 18.07224648, 18.09

In [10]:
y

array([68437., 68436., 68434., 68433., 68431., 68430., 68428., 68427.,
       68425., 68424., 68422., 68421., 68419., 68418., 68416., 68415.,
       68413., 68412., 68410., 68409., 68407., 68406., 68404., 68403.,
       68401., 68400., 68398., 68397., 68395., 68394., 68392., 68391.,
       68389., 68388., 68386., 68385., 68383., 68382., 68380., 68379.,
       68377., 68376., 68374., 68373., 68371., 68370., 68368., 68367.,
       68365., 68364., 68362., 68361., 68359., 68358., 68356., 68355.,
       68353., 68352., 68350., 68349., 68347., 68346., 68344., 68343.,
       68341., 68340., 68338., 68337., 68335., 68334., 68332., 68331.,
       68329., 68328., 68326., 68325., 68323., 68322., 68320., 68319.,
       68317., 68316., 68314., 68313., 68311., 68310., 68308., 68307.,
       68305., 68304., 68302., 68301., 68299., 68298., 68296., 68295.,
       68293., 68292., 68290., 68289., 68287., 68286., 68284., 68283.,
       68281., 68280., 68278., 68277., 68275., 68274., 68272., 68271.,
      